<a href="https://colab.research.google.com/github/fjtm/deep-fake-voice-recognition/blob/feature%2Fstart/04_testing_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# !git clone -b feature/start https://github.com/fjtm/deep-fake-voice-recognition.git
%cd deep-fake-voice-recognition/

[Errno 2] No such file or directory: 'deep-fake-voice-recognition/'
/content


In [23]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 10.5 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np

In [3]:
import torch
import pandas as pd

In [4]:
csv_files_path = f'/content/drive/My Drive/deep-fake-voice-recognition/data/'
train = pd.read_csv(csv_files_path+"train.csv")
test = pd.read_csv(csv_files_path+"test.csv")

In [5]:
target_column = "target"
index_columns = "ind_num"
not_data_columns = ["label", "ind", "target","ind_num"]

In [6]:
X_train = torch.tensor(train.drop(not_data_columns, axis = 1).values).type(torch.float)
y_train = torch.tensor(train[target_column].values).type(torch.float).view(-1, 1)
X_test = torch.tensor(test.drop(not_data_columns, axis = 1).values).type(torch.float)
y_test = torch.tensor(test[target_column].values).type(torch.float).view(-1, 1)
X_train.shape, y_train.shape, X_test.shape, y_test.shape
# Calculate the minimum and maximum values for each row
row_min = X_train.min(dim=0, keepdim=True).values
row_max = X_train.max(dim=0, keepdim=True).values

# Scale each row to the range [-1, 1]
X_train = 2 * (X_train - row_min) / (row_max - row_min) - 1

# Calculate the minimum and maximum values for each row
row_min = X_test.min(dim=0, keepdim=True).values
row_max = X_test.max(dim=0, keepdim=True).values

# Scale each row to the range [-1, 1]
X_test = 2 * (X_test - row_min) / (row_max - row_min) - 1

In [7]:
# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
import torch
import torch.nn as nn

class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(58, 112)  # Input size is 58, and the first hidden layer has 112 units
        self.layer_2 = nn.Linear(112, 1)  # 2 units to the output layer with 2 units for binary classification

    def forward(self, x):
        x = torch.relu(self.layer_1(x))
        x = self.layer_2(x)  # No activation function on the output for binary classification
        return x


DNN(
  (layer_1): Linear(in_features=58, out_features=112, bias=True)
  (layer_2): Linear(in_features=112, out_features=1, bias=True)
)

In [9]:
# Create a loss function
loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in

# Create an optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.1)

from torchmetrics.classification import BinaryF1Score
f1_metric = BinaryF1Score().to(device)

In [10]:
from torch.utils.data import TensorDataset, DataLoader
# Create a DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=10000, shuffle=False, num_workers=2)
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=10000, shuffle=False, num_workers=2)

In [11]:
from timeit import default_timer as timer
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format).
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

model_0 = DNN().to(device)

# Set the seed and start the timer
torch.manual_seed(42)
train_time_start_on_cpu = timer()

# Set the number of epochs (we'll keep this small for faster training times)
epochs = 3

# Create training and testing loop
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")
    ### Training
    train_loss = 0
    # Add a loop to loop through training batches
    for batch, (X, y) in enumerate(train_dataloader):
       # Send data to GPU
        X, y = X.to(device), y.to(device)
        model_0.train()
        # 1. Forward pass
        y_pred = model_0(X)

        # 2. Calculate loss (per batch)
        loss = loss_fn(y_pred, y)
        train_loss += loss # accumulatively add up the loss per epoch

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Print out how many samples have been seen
        if batch % 10 == 0:
            print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

    # Divide total train loss by length of train dataloader (average loss per batch per epoch)
    train_loss /= len(train_dataloader)

    ### Testing
    # Setup variables for accumulatively adding up loss and accuracy
    test_loss, test_f1 = 0, 0
    model_0.eval()
    with torch.inference_mode():
        for X, y in test_dataloader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            # 1. Forward pass
            test_pred = model_0(X)

            # 2. Calculate loss (accumatively)
            test_loss += loss_fn(test_pred, y) # accumulatively add up the loss per epoch

            # 3. Calculate accuracy (preds need to be same as y_true)
            test_f1 += f1_metric(target=y, preds=(test_pred >= 0.).float().view(-1, 1))

        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(test_dataloader)

        # Divide total accuracy by length of test dataloader (per batch)
        test_f1 /= len(test_dataloader)

    ## Print out what's happening
    print(f"\nTrain f1: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test f1: {test_f1:.2f}%\n")

# Calculate training time
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu,
                                           end=train_time_end_on_cpu,
                                           device=str(next(model_0.parameters()).device))